In [ ]:
from __future__ import print_function
import time
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

!pip install scikit-fuzzy
import skfuzzy as fuzz
from sklearn.cluster import MiniBatchKMeans

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer, Normalizer
from sklearn.model_selection import train_test_split

import nltk
from nltk.cluster import KMeansClusterer
from sklearn import cluster, metrics
nltk.download('punkt')
nltk.download('stopwords')

from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional

from sklearn.metrics import roc_auc_score
from scipy import interp
from sklearn.metrics import precision_recall_fscore_support

from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

import math
import tensorflow as tf


from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv


from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


from matplotlib import rc
from matplotlib import font_manager as fm, rcParams 
import matplotlib.pyplot as plt 
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)
plt.rcParams['figure.dpi'] = 120
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

#LatX fonts
rc('text', usetex=True)
plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']
!apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from keras.layers import Dense, Flatten, Dropout, LSTM, SpatialDropout1D, Conv1D, MaxPool1D, GRU, Input, Concatenate, GlobalMaxPooling1D
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from sklearn.decomposition import TruncatedSVD
!pip install dataframe_image
import dataframe_image as dfi
warnings.filterwarnings('ignore')
import nltk
from nltk import tokenize, ne_chunk
from nltk.stem import RSLPStemmer
from nltk.corpus import floresta, mac_morpho, masc_tagged
from pickle import load
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk import RegexpParser, Tree
from nltk.corpus import wordnet as wn
from nltk import RegexpParser
from nltk.stem import SnowballStemmer
from keras.initializers import Constant

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


def build_df(file_id, file_name, index_col=None):
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.GetContentFile(file_name)
  if index_col is not None :
    return pd.read_csv(file_name, index_col=index_col)
  return pd.read_csv(file_name)

df = build_df('1LEbg13-KBZzHmIWAv9SnthGUzjvcbJuA','stylo-data.csv')
news_df = build_df('15zZXsks6cI0FY4vpL555lBQSBr1QyjT4', 'news-data.csv' )
varela_df = build_df('1JoC7GlAmdQO9WPZeJZKMbv4djyQzeezC',  'varela-stylo-data.csv')
varela_df.pop('Subject')

#Some clean-up on line breaks
def clean_df(df):
  df = df.replace(to_replace ='\r', value = ' ', regex = True) 
  df = df.replace(to_replace ='\n ', value = '\n', regex = True) 
  df = df.replace(to_replace ='\n', value = ' ', regex = True)
  df = df.replace(to_replace ='\n{2,*}', value = ' ', regex = True)
  return df

#Read textual datasets
raw_df_test = build_df('1B-Ozck3R_cMeREw3yQQcODcz8UZ8Hvp7', 'raw_data.csv')
raw_df = clean_df(build_df('1B-Ozck3R_cMeREw3yQQcODcz8UZ8Hvp7', 'raw_data.csv'))
raw_news_df = clean_df(build_df('1P0UF5IRI3VNUuAulldEn84ntVfFLGiIq', 'raw_news_data.csv'))
raw_varela_df = clean_df(build_df('1s-qn2puLikOtN1YGOX2WZfZrNMxA6K4e', 'raw_varela_data.csv'))


stylo_classic_score_df = build_df('1g4zAbrxtVSqJ1GwwXKJufqMHXpWSrLsC', 'stylo_classic_score.csv', index_col=0)
tfidf_classic_score_df = build_df('17O4S-8P66KMzlKKReOcfsKO4LDDMSrYJ', 'idf_classic_score.csv', index_col=0)
raw_news_df.pop('Link')
raw_varela_df.pop('Subject')


def remove_varela_authors(df):
  regex_baleia = 'baleia - \d{1,2}\/\d{1,2}\/\d{2,4}'
  regex_ana = 'ana cristina cavalcante\s*\d{1,2} \w{1,4} \d{2,4} - \S{3,5}(min)*'
  regex_adriano = 'adriano gambarini - \d{1,2}\/\d{1,2}\/\d{2,4}'
  regex_ivolnildo = 'ivon(i)*l(d)*o lavôr(\s)*\d{1,2} \w{1,4} \d{2,4} - \S{4,5}(min)*'
  regex_mario = 'mário pinto(\s)*\d{1,2} \w{1,4} \d{2,4} - \S{4,5}(min)*'
  regex_julio = "mais sobr julio preuss - \d{1,2}\/\d{1,2}\/\d{2,4}(julio preussescreveu o livro 'fotografia digital: da compra da câmera à impressão das fotos)*"
  regex_roberto = 'roberto linsker - \d{1,2}\/\d{1,2}\/\d{2,4}'
  df['Text'] = df['Text'].str.replace(regex_baleia, ' ')
  df['Text'] = df['Text'].str.replace(regex_ana, ' ')
  df['Text'] = df['Text'].str.replace(regex_adriano, ' ')
  df['Text'] = df['Text'].str.replace(regex_ivolnildo, ' ')
  df['Text'] = df['Text'].str.replace(regex_mario, ' ')
  df['Text'] = df['Text'].str.replace(regex_julio, ' ')
  df['Text'] = df['Text'].str.replace(regex_roberto, ' ')
  return df

raw_varela_df = remove_varela_authors(raw_varela_df)

# Remove few class entries (We need at least 3 samples of an author) and encode Author column 

def remove_entries(df, size):
  return df.groupby('Author').filter(lambda x: len(x) > size)

def encode_target_column(df, column):
  le = LabelEncoder()
  y = df.pop(column)
  encoded_Y = le.fit_transform(y)
  return encoded_Y

df = remove_entries(df, 2)
raw_df = remove_entries(raw_df, 2)

encoded_Y = encode_target_column(df, 'Author')
encoded_raw_Y  = encode_target_column(raw_df, 'Author')

encoded_Y_news  = encode_target_column(news_df, 'Author')
raw_news_df.pop('Author')

varela_Y_backup = varela_df['Author']
encoded_varela_Y = encode_target_column(varela_df, 'Author')
raw_varela_df.pop('Author')


corpus = raw_df['Text']
corpus_news = raw_news_df['Text']
corpus_varela = raw_varela_df['Text']

In [ ]:
# Download word embedding for portuguese-BR (50/100 dim of several types)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

glove_50_link = 'https://drive.google.com/open?id=10MW2F53DYYxizSXLbvfsphs77Fu1U9Gz'
glove_50_id = '18TekAQRy2PxvHse_lfoL2Tt0pPcoptl-'

glove_100_link = 'https://drive.google.com/file/d/1GDsyY-TmS0gj_TyM3h_tnuBo9HBZk-80/view?usp=sharing'
glove_100_id = '1GDsyY-TmS0gj_TyM3h_tnuBo9HBZk-80'


w2vec_cbow_50 = 'https://drive.google.com/file/d/1h3JtxhSXvmRw4_qIpJ1jqhCR_aCEUfK7/view?usp=sharing'
w2vec_cbow_50_id = '1h3JtxhSXvmRw4_qIpJ1jqhCR_aCEUfK7'

w2vec_cbow_100 = 'https://drive.google.com/file/d/1H_PXtUi98MqFhpB4sWIt2J2J2ftjidj4/view?usp=sharing'
w2vec_cbow_100_id = '1H_PXtUi98MqFhpB4sWIt2J2J2ftjidj4'

fasttext_skip_50 = 'https://drive.google.com/file/d/1L2gR3GEb_od-ooxW-kaNuqXglnK4TMHc/view?usp=sharing'
fasttext_skip_50_id = '1L2gR3GEb_od-ooxW-kaNuqXglnK4TMHc'

fasttext_skip_100 = 'https://drive.google.com/file/d/1jm3cgIX7GyVmtH1Tr-IF5sxxfq0DT9qm/view?usp=sharing'
fasttext_skip_100_id = '1jm3cgIX7GyVmtH1Tr-IF5sxxfq0DT9qm'


def load_embedding(id, filename):
  downloaded = drive.CreateFile({'id':id})
  downloaded.GetContentFile(filename)
  word_embedding_dict = {}
  with open(filename,'r', encoding='utf-8') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)  
      coefs = np.fromstring(coefs, 'f', sep=' ')
      word_embedding_dict[word] = coefs
  return word_embedding_dict


glove_50_embedding = load_embedding(glove_50_id, 'glove_50')
glove_100_embedding = load_embedding(glove_100_id, 'glove_100')
w2vec_cbow_50_embedding = load_embedding(w2vec_cbow_50_id, 'w2v_50')
w2vec_cbow_100_embedding = load_embedding(w2vec_cbow_100_id, 'w2v_100')
fasttext_skip_50_embedding = load_embedding(fasttext_skip_50_id, 'ft_50')
fasttext_skip_100_embedding = load_embedding(fasttext_skip_100_id, 'ft_100')


In [ ]:
def build_matrix(corpus, word_dict):
  dim_size = len(word_dict.get('a'))
  tokenizer = Tokenizer(num_words=25000)
  tokenizer.fit_on_texts(corpus)
  # Convert strings to their corresponding integer value created by the Tokenizer
  embedded_sentences = tokenizer.texts_to_sequences(corpus)
  max_words_text = max(corpus, key=lambda sentence: len(nltk.word_tokenize(sentence, language='portuguese')))
  max_sentence_len = len(nltk.word_tokenize(max_words_text, language='portuguese'))
  padded_sentences = pad_sequences(embedded_sentences, max_sentence_len, padding='post')
  vocab_len = len(tokenizer.word_index) + 1

  # Create an embedding matrix, in other words, use the weights from gloVe vector against all our corpus
  # In the end we will have a matrix of all our vocabulary x 50 dimensions (size of glove dim)

  # initialize with all zeros
  embedding_matrix = np.zeros((vocab_len, dim_size))
  for word, index in tokenizer.word_index.items():
    embedding_vector = word_dict.get(word)
    # if the word is in gloVe, we will use its weigths 
    if embedding_vector is not None and embedding_vector.any():
      embedding_matrix[index] = embedding_vector[:dim_size]
    else:
      embedding_matrix[index] = np.random.randn(dim_size)
  return embedding_matrix

def build_matrixes(data, embedding):
  result = []
  for corpus in data:
    result.append(build_matrix(corpus, embedding))
  return result


student_glove_50, news_glove_50, varela_glove_50  = build_matrixes([corpus, corpus_news, corpus_varela], glove_50_embedding)
student_glove_100, news_glove_100, varela_glove_100  = build_matrixes([corpus, corpus_news, corpus_varela], glove_100_embedding)

student_w2v_50, news_w2v_50, varela_w2v_50  = build_matrixes([corpus, corpus_news, corpus_varela], w2vec_cbow_50_embedding)
student_w2v_100, news_w2v_100, varela_w2v_100  = build_matrixes([corpus, corpus_news, corpus_varela], w2vec_cbow_100_embedding)

student_ft_50, news_ft_50, varela_ft_50  = build_matrixes([corpus, corpus_news, corpus_varela], fasttext_skip_50_embedding)
student_ft_100, news_ft_100, varela_ft_100  = build_matrixes([corpus, corpus_news, corpus_varela], fasttext_skip_100_embedding)

In [ ]:
embedding_matrixes = [[student_ft_50, news_ft_50, varela_ft_50]
      [student_glove_50, news_glove_50, varela_glove_50],
      [student_glove_100, news_glove_100, varela_glove_100],
      [student_w2v_50, news_w2v_50, varela_w2v_50],
      [student_w2v_100, news_w2v_100, varela_w2v_100],
      [student_ft_50, news_ft_50, varela_ft_50],
      [student_ft_100, news_ft_100, varela_ft_100]]
      

In [ ]:
vocab_size = 10000
oov_token = "<OOV>"
max_length = 50
padding_type = "post"
trunction_type="post"

X_train, X_test , y_train, y_test = train_test_split(corpus, encoded_Y, test_size = 0.20)

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(corpus)

In [ ]:
max_features = 20000
def convert_corpus_to_number(corpus):
  tokenizer = Tokenizer(num_words=max_features)
  tokenizer.fit_on_texts(corpus)

  embedded_sentences = tokenizer.texts_to_sequences(corpus)

  longest_sentence = max(corpus, key=lambda sentence: len(nltk.word_tokenize(sentence, language='portuguese')))
  max_sentence_len = len(nltk.word_tokenize(longest_sentence, language='portuguese'))

  padded_sentences = pad_sequences(embedded_sentences, max_sentence_len, padding='post')

  return tokenizer, padded_sentences, max_sentence_len

tokenizer, padded_sentences, max_sent_size = convert_corpus_to_number(corpus)

word_index = tokenizer.word_index

In [ ]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

5527


In [ ]:
def split_and_train(corpus, encoded_Y, emd_matrix):
  tokenizer, padded_sentences, max_sent_size = convert_corpus_to_number(corpus)
  X_train, X_test , y_train, y_test = train_test_split(padded_sentences, encoded_Y, test_size = 0.30, random_state=0)
  y_train = to_categorical(y_train, len(set(encoded_Y)))
  y_test = to_categorical(y_test, len(set(encoded_Y)))
  return build_and_train_lstm(X_train, y_train , X_test, y_test, len(tokenizer.word_index) + 1, emd_matrix.shape[1], max_sent_size, emd_matrix, len(set(encoded_Y)))

def build_and_train_lstm(X_train, Y_train, X_test, Y_test, vocab_size, embd_size, max_sent_size, embd_matrix, num_classes, epochs=100):
  model = Sequential()
  embedding_layer = Embedding(vocab_size, embd_size, weights=[embd_matrix],
                              input_length=max_sent_size, trainable=False)
  model.add(embedding_layer)
  model.add(Dropout(0.2))
  model.add(Conv1D(64, 5, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))
  model.add(LSTM(16, return_sequences=True))
  model.add(Dropout(0.20))
  model.add(Flatten())
  model.add(Dense(num_classes))
  model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
  print('fitting model')
  history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=0)
  print('latest acc{} '.format(history.history['val_accuracy'][-1]))
  return max(history.history['val_accuracy'])

#split_and_train(corpus, encoded_Y, student_glove_50)
#build_and_train_lstm(X_train, y_train , X_test, y_test, len(tokenizer.word_index) + 1, student_glove_50.shape[1], max_sent_size, student_glove_50, len(set(encoded_Y)))


In [ ]:
def split_and_train_2(corpuses, Y, emd_matrix):
  results = {}
  #for idx, matrix in enumerate(emd_matrix):
  for i, corpus in enumerate(corpuses):
    acc = split_and_train(corpus, Y[i], emd_matrix[i])
    print(acc)
    results[i] = acc
    print(results)
  return results

split_and_train_2(
                [corpus, corpus_news, corpus_varela], 
                [encoded_Y, encoded_Y_news, encoded_varela_Y], 
                embedding_matrixes)

fitting model
latest acc0.07692307978868484 
0.07692307978868484
{0: 0.07692307978868484}
fitting model
latest acc0.20000000298023224 
0.30000001192092896
{0: 0.07692307978868484, 1: 0.30000001192092896}
fitting model
latest acc0.024444444105029106 
0.05444444343447685
{0: 0.07692307978868484, 1: 0.30000001192092896, 2: 0.05444444343447685}


{0: 0.07692307978868484, 1: 0.30000001192092896, 2: 0.05444444343447685}

In [ ]:
def split_and_train_2(corpuses, Y, emd_matrix):
  results = {}
  #for idx, matrix in enumerate(emd_matrix):
  for i, corpus in enumerate(corpuses):
    acc = split_and_train(corpus, Y[i], emd_matrix[i])
    print(acc)
    results[idx] = acc
    print(results)
  return results

split_and_train_2(
                [corpus, corpus_news, corpus_varela], 
                [encoded_Y, encoded_Y_news, encoded_varela_Y], 
                embedding_matrixes)

fitting model
latest acc0.11538461595773697 
{0: 0.11538461595773697}
fitting model
latest acc0.13333334028720856 
{0: 0.20000000298023224}
fitting model
latest acc0.036666665226221085 
{0: 0.04444444552063942}
fitting model
latest acc0.07692307978868484 
{0: 0.04444444552063942, 1: 0.07692307978868484}
fitting model
latest acc0.1666666716337204 
{0: 0.04444444552063942, 1: 0.23333333432674408}
fitting model
latest acc0.036666665226221085 
{0: 0.04444444552063942, 1: 0.03888889029622078}
fitting model
latest acc0.11538461595773697 
{0: 0.04444444552063942, 1: 0.03888889029622078, 2: 0.11538461595773697}
fitting model
latest acc0.10000000149011612 
{0: 0.04444444552063942, 1: 0.03888889029622078, 2: 0.2666666805744171}
fitting model
latest acc0.02222222276031971 
{0: 0.04444444552063942, 1: 0.03888889029622078, 2: 0.06666667014360428}
fitting model
latest acc0.07692307978868484 
{0: 0.04444444552063942, 1: 0.03888889029622078, 2: 0.06666667014360428, 3: 0.07692307978868484}
fitting mode

In [ ]:
embedding_matrixes = [
      [student_glove_50, news_glove_50, varela_glove_50],
      [student_glove_100, news_glove_100, varela_glove_100],
      [student_w2v_50, news_w2v_50, varela_w2v_50],
      [student_w2v_100, news_w2v_100, varela_w2v_100],
      [student_ft_50, news_ft_50, varela_ft_50],
      [student_ft_100, news_ft_100, varela_ft_100]
]

In [ ]:
num_words = min(vocab_size, len(word_index)) + 1
print(num_words)


embedding_dim = 50

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > vocab_size:
        continue
    embedding_vector = word_embedding_dict.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector[:embedding_dim]
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

5528


In [ ]:
X_train_padded = pad_sequences(tokenizer.texts_to_sequences(X_train),maxlen=len(tokenizer.word_index), padding=padding_type, truncating=trunction_type)
X_test_padded = pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen=len(tokenizer.word_index), padding=padding_type, truncating=trunction_type)

y_train = to_categorical(y_train, len(set(encoded_Y)))
y_test = to_categorical(y_test, len(set(encoded_Y)))

In [ ]:
from keras.initializers import Constant
model = Sequential()

model.add(Embedding(num_words,
                    50,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=sequence_length,
                    trainable=True))

model.add(Embedding(num_words,
                    50,
                    embeddings_initializer=Constant(student_glove_50),
                    input_length=sequence_length,
                    trainable=True))

model.add(LSTM(embedding_dim, return_sequences=True))

model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
embedding_layer = Embedding(len(word_index),
                            max_length,
                            weights=[student_glove_50],
                            input_length=6584,
                            trainable=False)

model = Sequential()

model.add(Embedding(num_words,
                    50,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=sequence_length,
                    trainable=True))

model.add(LSTM(embedding_dim, return_sequences=True))

model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
tokenizer.word_index

In [ ]:
def convert_docs_to_embeddings(corpus, word_dict, dim_size):
  tokenizer = Tokenizer(num_words=40000)
  tokenizer.fit_on_texts(corpus)
  
  max_words_text = max(corpus, key=lambda sentence: len(nltk.word_tokenize(sentence, language='portuguese')))
  max_sentence_len = len(nltk.word_tokenize(max_words_text, language='portuguese'))
  vocab_len = len(tokenizer.word_index) + 1

  docs = np.zeros((len(corpus), vocab_len, dim_size))
  
  for index, doc in enumerate(corpus):
    current_doc = np.zeros((vocab_len, dim_size))
    # NOTE: I realized that nltk.word_tokenize has a different behavior than split ''
    for word in doc.split(' '):
      embedding_vector = word_dict.get(word)
      if word.isalnum() and embedding_vector is not None and embedding_vector.any():
        idx = tokenizer.word_index[word]
        current_doc[idx] = embedding_vector[:dim_size]
    docs[index] = current_doc
  return docs
    

student_embedded_docs = convert_docs_to_embeddings(corpus, word_embedding_dict, 50)
#news_embedded_docs = convert_docs_to_embeddings(corpus_news, word_embedding_dict, 50)

# Crashing out of RAM
#varela_embedded_docs = convert_docs_to_embeddings(corpus_varela_sw, word_embedding_dict, 50)

In [ ]:
# cleaning up stopwords from Varela to see if memory can handle now
corpus_varela.shape

corpus_varela_sw = corpus_varela.apply(lambda x: [str(item) for item in x.split(' ') if item not in stopwords])
corpus_varela_sw = corpus_varela_sw.apply(lambda x: ' '.join(x))
corpus_varela_sw

## Model optimization

In [ ]:
# We have 7 classic models + 1 deep model
# Lets do GridSearchCV with each one + data normalization/scaling